Based one

https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma3_(4B).ipynb

In [1]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    #model_name = "unsloth/gemma-3-1b-pt-unsloth-bnb-4bit",
    model_name = "google/gemma-3-1b-pt",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    Quadro RTX 5000. Num GPUs = 1. Max memory: 16.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

## Load LoRA adapters

In [2]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [3]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [4]:
from datasets import load_dataset
dataset = load_dataset("philipfourie/books3_basic_sentenses_paraphrased-Morse", split="train")
dataset

Dataset({
    features: ['line', 'morse'],
    num_rows: 531976
})

In [5]:
dataset[0]

{'line': 'The title was cataloged by the Library of Congress.',
 'morse': '- .... . / - .. - .-.. . / .-- .- ... / -.-. .- - .- .-.. --- --. . -.. / -... -.-- / - .... . / .-.. .. -... .-. .- .-. -.-- / --- ..-. / -.-. --- -. --. .-. . ... ... .-.-.-'}

In [6]:
def apply_conversations(example):
    return {
        "conversations": [
            {
                "content": example["line"],
                "role": "user",
            },
            {
                "content": example["morse"] + tokenizer.eos_token,
                "role": "assistant",
            },
        ]
    }

In [7]:
convos = dataset.map(
    apply_conversations,
    remove_columns = ["line", "morse"],
    #num_proc = 1,
    desc = "Applying conversations",
)
convos

Applying conversations:   0%|          | 0/531976 [00:00<?, ? examples/s]

Dataset({
    features: ['conversations'],
    num_rows: 531976
})

In [8]:
convos[0]

{'conversations': [{'content': 'The title was cataloged by the Library of Congress.',
   'role': 'user'},
  {'content': '- .... . / - .. - .-.. . / .-- .- ... / -.-. .- - .- .-.. --- --. . -.. / -... -.-- / - .... . / .-.. .. -... .-. .- .-. -.-- / --- ..-. / -.-. --- -. --. .-. . ... ... .-.-.-<eos>',
   'role': 'assistant'}]}

In [9]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

convos = convos.map(formatting_prompts_func, remove_columns=["conversations"], batched = True)
convos

Map:   0%|          | 0/531976 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 531976
})

In [10]:
convos["text"][0]

'<start_of_turn>user\nThe title was cataloged by the Library of Congress.<end_of_turn>\n<start_of_turn>model\n- .... . / - .. - .-.. . / .-- .- ... / -.-. .- - .- .-.. --- --. . -.. / -... -.-- / - .... . / .-.. .. -... .-. .- .-. -.-- / --- ..-. / -.-. --- -. --. .-. . ... ... .-.-.-<eos><end_of_turn>\n'

In [29]:
# split the dataset into train and test
train_dataset = convos.train_test_split(test_size=0.1, seed=42)["train"]
test_dataset = convos.train_test_split(test_size=0.1, seed=42)["test"]

convos = None
dataset = None

AttributeError: 'NoneType' object has no attribute 'train_test_split'

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    args = SFTConfig(
        dataset_text_field="text",
        output_dir="outputs",  # Explicitly set output directory
        logging_dir="outputs/tensorboard_logs",  # Subdirectory for TensorBoard logs
        per_device_train_batch_size = 12,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!

        warmup_steps = 15,
        #num_train_epochs = 1.5, # Set this for 1 full training run.
        max_steps = 6200,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps=1,  # Log every step for testing
        eval_steps=1000,  # Enable evaluation logging
        eval_strategy="steps",
        
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "tensorboard", # Use this for WandB etc
        dataset_num_proc=4,
        
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/478778 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/53198 [00:00<?, ? examples/s]

In [31]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=12):   0%|          | 0/478778 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/53198 [00:00<?, ? examples/s]

In [32]:
trainer.train_dataset[100]["input_ids"]

[2,
 105,
 2364,
 107,
 37967,
 1135,
 496,
 44981,
 236752,
 657,
 506,
 109701,
 236761,
 106,
 107,
 105,
 4368,
 107,
 236772,
 4862,
 236761,
 2728,
 236772,
 753,
 965,
 9190,
 2728,
 24975,
 2728,
 24975,
 965,
 199900,
 965,
 199900,
 856,
 2728,
 753,
 753,
 199900,
 856,
 965,
 199900,
 753,
 965,
 753,
 27103,
 783,
 965,
 783,
 24975,
 2728,
 3729,
 753,
 199900,
 4862,
 4862,
 1,
 106,
 107]

In [33]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, "X")

'XXXXXXXXXXXXXXXXXX-.-. ..- - / --- ..-. ..-. / .- / .-.. .. - - .-.. / .- - / - .... . / .-. .. ... - .-.-.-<eos><end_of_turn>\n'

In [34]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Quadro RTX 5000. Max memory = 16.0 GB.
7.797 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train(resume_from_checkpoint = True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 478,778 | Num Epochs = 2 | Total steps = 14,961
O^O/ \_/ \    Batch size per device = 12 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (12 x 4 x 1) = 48
 "-____-"     Trainable parameters = 6,522,880/1,006,408,832 (0.65% trained)


Step,Training Loss,Validation Loss
2000,0.068600,0.089240
4000,0.028000,0.033172


Unsloth: Not an error, but Gemma3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
/home/philipf/dev/testj/.venv/lib/python3.12/site-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 362c32f2-a5b4-4b8d-8fb7-1caf1b3211e9)') - silently ignoring the lookup for the file config.json in unsloth/gemma-3-1b-pt.
  warnings.warn(
/home/philipf/dev/testj/.venv/lib/python3.12/site-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in unsloth/gemma-3-1b-pt - will assume that the vocabulary was not modified.
  warnings.warn(


KeyboardInterrupt: 

In [55]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Hi Phil, can you please send food and water as soon as possible?",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize= False,
    add_generation_prompt = True, # Must add for generation
)


outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 200, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1, top_p = 0.95, top_k = 64,
)

output  = tokenizer.batch_decode(outputs)[0]
print(output)

from decode import decode_from_morse

import re

match = re.search(r'<start_of_turn>model\n(.*?)<eos>', output, re.DOTALL)
if match:
    result = match.group(1)
    #print(result)
    print(decode_from_morse(result))
else:
    print("No match found.")
    

<bos><bos><start_of_turn>user
Hi Phil, can you please send food and water as soon as possible?<end_of_turn>
<start_of_turn>model
.... .. / .--. .... .. .-.. --..-- / -.-. .- -. / -.-- --- ..- / .--. .-.. . .- ... . / ... . -. -.. / ..-. --- --- -.. / .- -. -.. / .-- .- - . .-. / .- ... / ... --- --- -. / .- ... / .--. --- ... ... .. -... .-.. . ..--..<eos>
HI PHIL, CAN YOU PLEASE SEND FOOD AND WATER AS SOON AS POSSIBLE?


In [ ]:
#model.save_pretrained("gemma-3-1b-sos")  # Local saving
#tokenizer.save_pretrained("gemma-3-1b-sos")

In [ ]:
if False: # Change to True to save finetune!
    #model.save_pretrained_merged("gemma-3-finetune2", tokenizer, save_method = "merged_4bit")
    #odel.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit",)
    #odel.save_pretrained_gguf("dir", tokenizer, quantization_method = "Q8_0")
    model.push_to_hub_gguf("philipfourie/gemma-3-finetune",  quantization_type ="q8_0")

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "philipfourie/gemma-3-1b-morse-code", tokenizer,
    )

In [ ]:
# model.save_pretrained_gguf(
#     "gemma-3-1b-morse-code",
#     quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
# )

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        #"gemma-3-1b-morse-code",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "philipfourie/gemma-3-1b-morse-code"
    )

In [56]:
if True: # Change to True to save finetune!
    model.save_pretrained_merged("morse-code", tokenizer)

Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:33<00:00, 33.02s/it]


In [57]:
if True: # Change to True to upload finetune
    model.push_to_hub_merged(
        "philipfourie/morse-code", 
        tokenizer,
    )

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.
Unsloth: Merging weights into 16bit:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:50<00:00, 50.94s/it]


In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "morse-code",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Unsloth: llama.cpp folder already exists - will use `llama.cpp`


NotImplementedError: Unsloth: llama.cpp GGUF conversion does not yet support converting model types of `Gemma3ForCausalLM`.

In [58]:
if True: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "morse-code",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "philipfourie/morse-code_q8"
    )

NotImplementedError: Unsloth: llama.cpp GGUF conversion does not yet support converting model types of `Gemma3ForCausalLM`.